# Download historical equity data for NASDAQ stocks from yahoo finance

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from time import time
from tqdm import tqdm
from pathlib import Path
import pandas as pd

from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
import yfinance as yf

In [3]:
idx = pd.IndexSlice

In [4]:
results_path = Path('results', 'asset_pricing')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [5]:
def chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [6]:
def format_time(t):
    """Return a formatted time string 'HH:MM:SS
    based on a numeric time() value"""
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return f'{h:0>2.0f}:{m:0>2.0f}:{s:0>2.0f}'

## Get NASDAQ symbols

In [17]:
traded_symbols = get_nasdaq_symbols()

In [19]:
traded_symbols[~traded_symbols.ETF]


,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
AACG,True,ATA Creativity Global - American Depositary Sh...,Q,G,False,100.0,False,D,NaN,AACG,False
AACI,True,Armada Acquisition Corp. I - Common Stock,Q,G,False,100.0,False,N,NaN,AACI,False
AACIU,True,Armada Acquisition Corp. I - Unit,Q,G,False,100.0,False,N,NaN,AACIU,False
...,...,...,...,...,...,...,...,...,...,...,...
ZXIET,True,IEX Test Company Test Symbol Three for IEX,V,,False,100.0,True,NaN,ZXIET,ZXIET,False
ZXYZ.A,True,Nasdaq Symbology Test Common Stock,Q,Q,False,100.0,True,N,NaN,ZXYZ.A,False
ZXZZT,True,NASDAQ TEST STOCK,Q,G,False,100.0,True,N,NaN,ZXZZT,False


In [8]:
traded_symbols.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11145 entries, A to ZZZ
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Nasdaq Traded     11145 non-null  bool    
 1   Security Name     11145 non-null  object  
 2   Listing Exchange  11145 non-null  category
 3   Market Category   11145 non-null  object  
 4   ETF               11145 non-null  bool    
 5   Round Lot Size    11145 non-null  float64 
 6   Test Issue        11145 non-null  bool    
 7   Financial Status  4982 non-null   category
 8   CQS Symbol        6163 non-null   object  
 9   NASDAQ Symbol     11144 non-null  object  
 10  NextShares        11145 non-null  bool    
dtypes: bool(4), category(2), float64(1), object(4)
memory usage: 588.1+ KB


## Download metadata from yahoo finance

### NASDAQ symbols

In [9]:
all_symbols = traded_symbols[~traded_symbols.ETF].index.unique().to_list()
n = len(all_symbols)
print(f'# Symbols: {n:,.0f}')

# Symbols: 7,774


In [10]:
all_symbols = [item for item in all_symbols if not isinstance(item, float)]

In [12]:
yf_symbols = yf.Tickers(all_symbols)

In [41]:
meta_data = []
start = time()
for ticker, yf_object in tqdm(yf_symbols.tickers.items()):
    try:
        s = pd.Series(yf_object.get_info())
        meta_data.append(s.to_frame(ticker))
    except Exception as e:
        # track errors
        print(symbol.ticker, e)
        
print(f'Success: {len(meta_data):5,.0f} / {n:5,.0f}')

  0%|          | 7/7775 [00:00<13:26,  9.63it/s]404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AACT.U?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AACT.U&crumb=Vn1kjkpqGe7
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AACT.W?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AACT.W&crumb=Vn1kjkpqGe7
  2%|▏         | 157/7775 [00:20<10:15, 12.37it/s]404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AEL$A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AEL%24A&crumb=Vn1kjkpqGe7
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AEL$B?

Success: 7,775 / 7,776


In [42]:
df = pd.concat(meta_data, axis=1).dropna(how='all').T
df = df.apply(pd.to_numeric, errors='ignore')
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 7775 entries, A to ZYXI
Columns: 146 entries, address1 to industrySymbol
dtypes: float64(111), object(35)
memory usage: 8.7+ MB


In [44]:
df.to_hdf(results_path / 'data.h5', 'stocks/info')

## Download adjusted price data using yfinance

In [11]:
prices_adj = []
start = time()
for i, chunk in enumerate(chunks(all_symbols, 100), 1):
    prices_adj.append(yf.download(chunk, start='1989-12-01', end='2020-1-31', auto_adjust=True).stack(-1))

    per_ticker = (time()-start) / (i * 100)
    to_do = n - (i * 100)
    to_go = to_do * per_ticker    
    print(f'Success: {len(prices_adj):5,.0f}/{i:5,.0f} | To go: {format_time(to_go)} ({to_do:5,.0f})')

[*********************100%%**********************]  100 of 100 completed

49 Failed downloads:
['ACAH', 'ACHR', 'ABOS', 'ACON', 'ACBA', 'ACLX', 'ACI', 'ABLV', 'ABL', 'ACHL', 'ACAB', 'AACT', 'ACAC', 'ACACU', 'ACABU', 'ACCD', 'ABVX', 'ABSI', 'ABCL', 'ACRV', 'ABNB', 'AACI', 'ACBAU', 'ACAHU', 'ACGLN', 'AACIU', 'ABLLL', 'AAGR', 'ACDC', 'AAN', 'AAMC']: Exception("%ticker%: Data doesn't exist for startDate = 628491600, endDate = 1580446800")
['ABLLW', 'ACBAW', 'AACIW', 'ABLVW', 'ACABW', 'ACACW', 'ACONW', 'ACAHW', 'AAGRW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['ABR$F', 'ACR$D', 'AACT.W', 'ACP$A', 'ABR$E', 'ACR$C', 'ACHR.W', 'ABR$D', 'AACT.U']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*                      2%%                      ]  2 of 100 completed

Success:     1/    1 | To go: 00:19:57 (7,674)


[*********************100%%**********************]  100 of 100 completed

49 Failed downloads:
['AEAEW', 'ADOCR', 'AFARW', 'AERTW', 'AENTW', 'ADVWW', 'ADNWW', 'AFRIW', 'ADSEW', 'ADTHW', 'ADOCW', 'AFJKR']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['AESI', 'AESC', 'AFRI', 'AEVA', 'ADSE', 'AFIB', 'AFCG', 'AERT', 'ACXP', 'AEAEU', 'AFJKU', 'AEI', 'AFGB', 'ADRT', 'AFGE', 'AFARU', 'ACT', 'ADXN', 'AFGD', 'AFJK', 'AENT', 'ADCT', 'AEON', 'ACVA', 'AFRM', 'ADAG', 'ADOC', 'ADTH', 'ADTX', 'AEAE', 'AFAR']: Exception("%ticker%: Data doesn't exist for startDate = 628491600, endDate = 1580446800")
['AEL$B', 'ADRT.U', 'AEON.W', 'AEL$A', 'ADC$A', 'AEVA.W']: Exception('%ticker%: No timezone found, symbol may be delisted')


Success:     2/    2 | To go: 00:19:20 (7,574)


[*********************100%%**********************]  100 of 100 completed

50 Failed downloads:
['AGNCP', 'AGNCL', 'AKA', 'AIRE', 'AIRC', 'AIP', 'AIXI', 'AITRU', 'AITR', 'AHI', 'AIMD', 'AIMBU', 'AIB', 'AGFY', 'AIRS', 'AGRI', 'AIMAU', 'AIBBU', 'AGTI', 'AKLI', 'AIZN', 'AKAN', 'AI', 'AISP', 'AGL']: Exception("%ticker%: Data doesn't exist for startDate = 628491600, endDate = 1580446800")
['AGM$D', 'AGM$E', 'AGM.A', 'AKO.B', 'AHT$G', 'AGM$G', 'AHL$C', 'AHT$D', 'AHH$A', 'AHT$F', 'AHL$E', 'AGM$F', 'AHT$H', 'AKO.A', 'AGM$C', 'AHL$D', 'AIG$A', 'AHT$I']: Exception('%ticker%: No timezone found, symbol may be delisted')
['AIMAW', 'AGRIW', 'AGBAW', 'AISPW', 'AITRR', 'AIMDW', 'AIBBR']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
[*                      3%%                      ]  3 of 100 completed

...

Success:    76/   76 | To go: 00:00:22 (  174)


[*********************100%%**********************]  100 of 100 completed

39 Failed downloads:
['XPDBW', 'WTMAR', 'XFINW', 'YOTAR', 'XOSWW', 'XBPEW', 'YOTAW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['XPDB', 'WTO', 'XELAP', 'XPER', 'YOU', 'YALA', 'XOS', 'XOMAO', 'XLO', 'WTMA', 'XPOF', 'YQ', 'XBP', 'XMTR', 'YMM', 'YGMZ', 'XOMAP', 'WUCT', 'XPON', 'YS', 'XPEV', 'YOTAU', 'XFINU', 'XRTX', 'XFIN', 'YGF', 'YOSH', 'WTMAU', 'XPDBU', 'YOTA']: Exception("%ticker%: Data doesn't exist for startDate = 628491600, endDate = 1580446800")
['YCBD$A', 'XFLT$A']: Exception('%ticker%: No timezone found, symbol may be delisted')
[**                     4%%                      ]  3 of 73 completed

Success:    77/   77 | To go: 00:00:09 (   74)


[*********************100%%**********************]  73 of 73 completed

41 Failed downloads:
['YSBPW', 'ZCARW', 'ZBZZT', 'ZCZZT', 'ZAZZT', 'ZAPPW', 'ZXZZT', 'ZLSWW', 'ZVV', 'ZURAW', 'ZVZZT', 'ZWZZT', 'ZJZZT']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['ZIM', 'ZLSWU', 'ZETA', 'YSG', 'ZENV', 'ZVSA', 'ZFOXW', 'ZLS', 'ZGN', 'ZCAR', 'ZNTL', 'ZVIA', 'ZIMV', 'ZKH', 'ZAPP', 'ZCMD', 'ZFOX', 'ZI', 'ZIP', 'ZH', 'ZJYL', 'ZURA']: Exception("%ticker%: Data doesn't exist for startDate = 628491600, endDate = 1580446800")
['ZXYZ.A', 'ZEXIT', 'ZXIET', 'ZIEXT', 'ZBZX', 'ZTEST']: Exception('%ticker%: No timezone found, symbol may be delisted')


Success:    78/   78 | To go: -1:59:57 (  -26)


In [12]:
prices_adj = (pd.concat(prices_adj)
              .dropna(how='all', axis=1)
              .rename(columns=str.lower)
              .swaplevel())

In [13]:
prices_adj.index.names = ['ticker', 'date']

In [14]:
len(prices_adj.index.unique('ticker'))

4615

In [15]:
prices_adj.to_hdf(results_path / 'data.h5', 'stocks/prices/adjusted_temp')

In [6]:
prices_adj = pd.read_hdf(results_path / 'data.h5', 'stocks/prices/adjusted_temp')

### Remove outliers

In [7]:
df = prices_adj.close.unstack('ticker')
pmax = df.pct_change().max()
pmin = df.pct_change().min()
to_drop = pmax[pmax > 1].index.union(pmin[pmin<-1].index)
len(to_drop)

555

In [15]:
len(prices_adj.index.unique('ticker'))

4060

In [16]:
prices_adj.sort_index().loc[idx[:, '1990': '2019'], :].to_hdf(results_path / 'data.h5', 
                                                              'stocks/prices/adjusted')